# Data exploration

# Download Zooniverse annotations, build db EDIT: Actually don't need this

In [1]:
import io
import json
from multiprocessing import AuthenticationError

import pandas as pd
import numpy as np

from panoptes_client import Panoptes, Project
from pandas.api.types import is_object_dtype

In [1]:
# zoo_user = getpass.getpass("Enter your Zooniverse user")
zoo_user = "lioruzan"
# zoo_pass = getpass.getpass("Enter your Zooniverse password")
zoo_pass = "@vsYP1muGZ3Z1R&I"


# Connect to Zooniverse with your username and password
auth = Panoptes.connect(username=zoo_user, password=zoo_pass)

if not auth.logged_in:
    raise AuthenticationError("Your credentials are invalid. Please try again.")

# Connect to the Zooniverse project (our frog project # is 13355)
project = Project(13355)

# Get the export classifications
export = project.get_export("classifications")

# Save the response as pandas data frame
classifications = pd.read_csv(
    io.StringIO(export.content.decode("utf-8")),
)
classifications.to_excel("zooniverse_keypoint_annotations.xls")
classifications.to_csv("zooniverse_keypoint_annotations.csv")

# for operational sake keep just these columns
classifications = classifications[["subject_data", "annotations"]]

/var/folders/s9/12gqnpd134j4ft6c49pfgsgr0000gn/T/ipykernel_59450/3379051029.py:32: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  classifications.to_excel("zooniverse_keypoint_annotations.xls")


In [2]:
# load rows from disk
classifications = pd.read_csv("zooniverse_keypoint_annotations.csv", usecols=["subject_data", "annotations"])

In [3]:
def get_subject_data(subject_data_string):
    return list(json.loads(subject_data_string).values())[0]

In [4]:
# filter rows with bad capture ids (2-3 tops)
classifications_df = (
    classifications.join(
    pd.DataFrame(
        data=classifications.subject_data.map(get_subject_data).to_list()))
    .drop(columns=["subject_data", "retired", "frog_id"])
)

# filter nans in "capture" column
idx = classifications_df.capture.notna()
classifications_df = classifications_df[idx]

# filter entries that aren't numeric then cast to int32
numeric_idx = classifications_df.capture.str.isnumeric()
classifications_df = classifications_df[numeric_idx].astype({"capture": np.int32})

In [5]:
classifications_df.columns

Index(['annotations', 'grid', 'capture', 'filename'], dtype='object')

In [56]:
from dateutil.parser import parse
from datetime import datetime
from typing import Any

def is_date(obj: Any) -> bool:
    """
    Return whether the string can be interpreted as a date.

    :param obj: object suspected of being a date
    """
    if obj is np.nan or obj is None:
        return False

    else:
        if isinstance(obj, datetime):
            return True
        
        try: 
            parse(obj, fuzzy=True)
            return True

        except ValueError:
            return False

    

In [65]:
def is_number(obj: Any) -> bool:
    if (obj is np.nan) or (obj is np.inf) or (obj is -np.inf):
        return False
    elif isinstance(obj, int):
        return True
    elif isinstance(obj, float):
        return True
    else:
        return False

In [70]:
# filter out all bad dates and capture numbers
capture_idx = df.loc[:, "Capture #"].apply(is_number)
date_idx = df.loc[:, "Date"].apply(is_date)
idx = capture_idx & date_idx
print(sum(idx), sum(~idx))
df2 = df[idx].reset_index(drop=True)
print(df2)
df2["Test"] = "test"
df2

829 1209
                    Date Capture # Bag # Time Found  \
0    2005-11-07 00:00:00         1    39   10:19:00   
1    2005-11-07 00:00:00         2     5   10:27:00   
2    2005-11-07 00:00:00         3    17   10:25:00   
3    2005-11-07 00:00:00         4     1   09:39:00   
4    2005-11-07 00:00:00         5     2   09:46:00   
..                   ...       ...   ...        ...   
824  2020-11-26 00:00:00       825   NaN        NaN   
825  2020-11-26 00:00:00       826   NaN        NaN   
826  2020-11-26 00:00:00       827   NaN        NaN   
827  2020-11-26 00:00:00       828   NaN        NaN   
828  2020-11-26 00:00:00       829   NaN        NaN   

    Capture Location (to nearest m2) X axis Y axis        Habitat Type  \
0                                 H7    NaN    NaN          rice grass   
1                                 H5    NaN    NaN          rice grass   
2                                 J7    NaN    NaN    hen&chicken fern   
3                                 

,Date,Capture #,Bag #,Time Found,Capture Location (to nearest m2),X axis,Y axis,Habitat Type,Height above ground (cm),Searcher,...,Back right mark,Face left mark,Face right mark,Capture photo code,Frog ID #,Recapture from Previous Session,Within Session Recapture,ID Notes,ID From photo,Test
0,2005-11-07 00:00:00,1,39,10:19:00,H7,NaN,NaN,rice grass,5,H. Cooper,...,1,0,0,1100-1,A1,no,no,NaN,NaN,test
1,2005-11-07 00:00:00,2,5,10:27:00,H5,NaN,NaN,rice grass,15,H. Cooper,...,0,0,0,0000-2,A2,no,no,NaN,NaN,test
2,2005-11-07 00:00:00,3,17,10:25:00,J7,NaN,NaN,hen&chicken fern,2,A. Smale,...,0?,0,0,1_00-3,A3,no,no,1493 better image,NaN,test
3,2005-11-07 00:00:00,4,1,09:39:00,E1,NaN,NaN,hen&chicken fern,10,A. Smale,...,1,0,1,0101-4,A4,no,no,left nare stripe,NaN,test
4,2005-11-07 00:00:00,5,2,09:46:00,B3,NaN,NaN,on leaf of sapling,20,W. Davies,...,1,0,0,0100-5,A5,no,no,very brick coloured and fat,NaN,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824,2020-11-26 00:00:00,825,NaN,NaN,H4,143,167,fern,NaN,Maadi,...,0,0,0,8250000,A201,yes,yes,NaN,NaN,test
825,2020-11-26 00:00:00,826,NaN,NaN,J5,50,500,ponga,150,Troy,...,0,0,0,8260000,A216,yes,no,Possibly 0000,NaN,test
826,2020-11-26 00:00:00,827,NaN,NaN,B9,90,890,ponga,90,Troy,...,1,0,0,8271100,A200,yes,yes,NaN,NaN,test
827,2020-11-26 00:00:00,828,NaN,NaN,I2,20,190,Ground,0,Troy,...,0,0,0,8280000,A197,yes,yes,"""Ninja"" (25/11/2020 & 26/11/2020)",NaN,test


In [71]:
import matplotlib.pyplot as plt
whareorino_excel_file = f"Whareorino frog monitoring data 2005 onwards CURRENT FILE - DOCDM-106978.xls"
grids = ["Grid A", "Grid B", "Grid C", "Grid D"]

# for grid in grids:
#     df = pd.read_excel(whareorino_excel_file, sheet_name=grid)
#     date_idx = df.loc[:, "Date"].apply(is_date)
#     print(grid, df.loc[:, "Date"])
    
big_df = pd.read_excel(whareorino_excel_file, sheet_name=grids)
big_df

{'Grid A':                      Date Capture # Bag # Time Found  \
 0     2005-11-07 00:00:00         1    39   10:19:00   
 1     2005-11-07 00:00:00         2     5   10:27:00   
 2     2005-11-07 00:00:00         3    17   10:25:00   
 3     2005-11-07 00:00:00         4     1   09:39:00   
 4     2005-11-07 00:00:00         5     2   09:46:00   
 ...                   ...       ...   ...        ...   
 2033                  NaN      1996   NaN        NaN   
 2034                  NaN      1997   NaN        NaN   
 2035                  NaN      1998   NaN        NaN   
 2036                  NaN      1999   NaN        NaN   
 2037                  NaN      2000   NaN        NaN   
 
      Capture Location (to nearest m2) X axis Y axis        Habitat Type  \
 0                                  H7    NaN    NaN          rice grass   
 1                                  H5    NaN    NaN          rice grass   
 2                                  J7    NaN    NaN    hen&chicken fern   


In [6]:
""" open whareorino excel and try to connect each row to one or more pictures """
dbs = dict()
whareorino_excel_file = f"Whareorino frog monitoring data 2005 onwards CURRENT FILE - DOCDM-106978.xls"
grids = ["Grid A", "Grid B", "Grid C", "Grid D"]

for grid in grids:
    df = pd.read_excel(whareorino_excel_file, sheet_name=grid)

    # filter nans and cast Capture # as int32
    if is_object_dtype(df["Capture #"]):
        int_idx = df["Capture #"].str.isnumeric().isna()
        df = df[int_idx]
        int_idx = df["Capture #"].notna()
        dbs[grid] =  df[int_idx].astype({"Capture #": np.int32})
    
    else:
        nan_idx = df["Capture #"].notna()
        dbs[grid] = df[nan_idx].astype({"Capture #": np.int32})

In [7]:
pukeokahu_excel_file = "Pukeokahu Monitoring Data 2006 onwards - DOCDM-95563.xls"

df = pd.read_excel(pukeokahu_excel_file, sheet_name="MR Data")

# filter nans and cast Capture # as int32
notna_idx = df["Capture #"].notna()
dbs['Pukeokahu Frog Monitoring'] = (
    df[notna_idx]
    .astype({"Capture #": np.int32})
)

In [8]:
db = pd.concat(list(dbs.values())).reset_index(drop=True)
db

,Date,Capture #,Bag #,Time Found,Capture Location (to nearest m2),X axis,Y axis,Habitat Type,Height above ground (cm),Searcher,...,Back left mark,Back right mark,Face left mark,Face right mark,Capture photo code,Frog ID #,Recapture from Previous Session,Within Session Recapture,ID Notes,ID From photo
0,2005-11-07 00:00:00,1,39,10:19:00,H7,NaN,NaN,rice grass,5,H. Cooper,...,1,1,0,0,1100-1,A1,no,no,NaN,NaN
1,2005-11-07 00:00:00,2,5,10:27:00,H5,NaN,NaN,rice grass,15,H. Cooper,...,0,0,0,0,0000-2,A2,no,no,NaN,NaN
2,2005-11-07 00:00:00,3,17,10:25:00,J7,NaN,NaN,hen&chicken fern,2,A. Smale,...,1,0?,0,0,1_00-3,A3,no,no,1493 better image,NaN
3,2005-11-07 00:00:00,4,1,09:39:00,E1,NaN,NaN,hen&chicken fern,10,A. Smale,...,0,1,0,1,0101-4,A4,no,no,left nare stripe,NaN
4,2005-11-07 00:00:00,5,2,09:46:00,B3,NaN,NaN,on leaf of sapling,20,W. Davies,...,0,1,0,0,0100-5,A5,no,no,very brick coloured and fat,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10477,2020-11-12 00:00:00,8897,55,22:00:00,H25,H36,2496,ponga,5,Emily E,...,1,1,0,0,1100-8897,P497,no,no,NaN,NaN
10478,2020-11-12 00:00:00,8898,31,21:52:00,P23,P70,2276,fern fronds,0,Kelly-Anne,...,1,1,0,0,1100-8898,P498,no,no,NaN,NaN
10479,2020-11-12 00:00:00,8899,52,22:25:00,G14,G58,1394,BlEflu,7,Emily E,...,1,1,0,0,1100-8899,P492,no,no,NaN,NaN
10480,2020-11-12 00:00:00,8900,40,22:03:00,I20,I73,1960,fern,10,Emily J,...,1,1,0,1,1101-8890,P493,no,no,NaN,NaN


In [9]:
# use "capture", "Capture #" columns to merge rows from Zooniverse data to excel db rows
db = db.merge(right=classifications_df, left_on="Capture #", right_on="capture").drop(columns=["capture"])
db

,Date,Capture #,Bag #,Time Found,Capture Location (to nearest m2),X axis,Y axis,Habitat Type,Height above ground (cm),Searcher,...,Face right mark,Capture photo code,Frog ID #,Recapture from Previous Session,Within Session Recapture,ID Notes,ID From photo,annotations,grid,filename
0,2005-11-07 00:00:00,1,39,10:19:00,H7,NaN,NaN,rice grass,5,H. Cooper,...,0,1100-1,A1,no,no,NaN,NaN,"[{""task"":""T0"",""task_label"":""Label the landmark...",Grid A,1100-1.jpg
1,2005-11-07 00:00:00,1,39,10:19:00,H7,NaN,NaN,rice grass,5,H. Cooper,...,0,1100-1,A1,no,no,NaN,NaN,"[{""task"":""T0"",""task_label"":""Label the landmark...",Grid A,1100-1.jpg
2,2005-11-07 00:00:00,2,5,10:27:00,H5,NaN,NaN,rice grass,15,H. Cooper,...,0,0000-2,A2,no,no,NaN,NaN,"[{""task"":""T0"",""task_label"":""Label the landmark...",Grid C,1101-2.jpg
3,2005-11-07 00:00:00,2,5,10:27:00,H5,NaN,NaN,rice grass,15,H. Cooper,...,0,0000-2,A2,no,no,NaN,NaN,"[{""task"":""T0"",""task_label"":""Label the landmark...",Grid A,0000-2.jpg
4,2005-11-07 00:00:00,5,2,09:46:00,B3,NaN,NaN,on leaf of sapling,20,W. Davies,...,0,0100-5,A5,no,no,very brick coloured and fat,NaN,"[{""task"":""T0"",""task_label"":""Label the landmark...",Grid A,0100-5.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2276,2018-11-15 00:00:00,8785,68,21:40:00,G20,G35,1950,blec fluv,1,Leigh,...,1,1101-8785,P388,yes,yes,Caught 14/11/18,NaN,"[{""task"":""T0"",""task_label"":""Label the landmark...",Pukeokahu Frog Monitoring,1101-8785.JPG
2277,2018-11-15 00:00:00,8791,61,21:35:00,H21,H76,2000,blec fluv,1,Leigh,...,0,1100-8791,P452,no,no,new,NaN,"[{""task"":""T0"",""task_label"":""Label the landmark...",Pukeokahu Frog Monitoring,1100-8791.JPG
2278,2018-11-15 00:00:00,8792,27,22:14:00,R21,R15,2011,ponga leaf litter,0,Abi,...,0,1100-8792,P453,no,no,new,NaN,"[{""task"":""T0"",""task_label"":""Label the landmark...",Pukeokahu Frog Monitoring,1100-8792.JPG
2279,2018-11-15 00:00:00,8796,51,22:45:00,Q19,Q53,1883,fern frond,0,Kat,...,0,1_00-8796,P456,no,no,new,NaN,"[{""task"":""T0"",""task_label"":""Label the landmark...",Pukeokahu Frog Monitoring,1_00-8796.JPG


## Concatenate all grid dataframes to one, use "grid" column to differentiate them

In [10]:
db.to_excel("27.11.22_capture_db.xls", index=False)

/var/folders/s9/12gqnpd134j4ft6c49pfgsgr0000gn/T/ipykernel_48073/4189465488.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  db.to_excel("27.11.22_capture_db.xls", index=False)
